In [1]:
from datasets import load_dataset

ds = load_dataset("roneneldan/TinyStories")

In [3]:
print(ds["train"][0])

{'text': 'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.'}


In [ ]:
import os
from transformers import MarianMTModel, MarianTokenizer


model_id = "Helsinki-NLP/opus-mt-en-ru"
save_dir = "./opus-mt-en-ru"
os.makedirs(save_dir, exist_ok=True)


print("📥 Скачиваем модель и токенайзер...")
tokenizer = MarianTokenizer.from_pretrained(model_id, cache_dir=save_dir)
model = MarianMTModel.from_pretrained(model_id, cache_dir=save_dir)

print("💾 Сохраняем в:", save_dir)
tokenizer.save_pretrained(save_dir)
model.save_pretrained(save_dir)



📥 Скачиваем модель и токенайзер...


c:\Users\Пользоваткль\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
c:\Users\Пользоваткль\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


💾 Сохраняем в: ./opus-mt-en-ru


c:\Users\Пользоваткль\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62517]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [ ]:

print("\n📘 Пример перевода:")
sample_text = ["Hello, how are you"]
inputs = tokenizer(sample_text, return_tensors="pt", padding=True, truncation=True)
translated = model.generate(**inputs)
output = tokenizer.batch_decode(translated, skip_special_tokens=True)

print("EN:", sample_text[0])
print("RU:", output[0])


📘 Пример перевода:
EN: Привет, как дела?
RU: Понимаешь, Кек Телла?


In [ ]:
from datasets import load_dataset, Dataset
import torch
from tqdm import tqdm



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

dataset = load_dataset("roneneldan/TinyStories", split="train[:1000]")

translated_data = []

print("🔁 Переводим истории...")
for example in tqdm(dataset, total=1000):
    text = example["text"]
    inputs = tokenizer([text], return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    translated_tokens = model.generate(**inputs, max_length=512)
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

    translated_data.append({
        "original": text,
        "translation": translated_text
    })


new_dataset = Dataset.from_list(translated_data)


print("💾 Сохраняем в файл: translated_tiny_stories.jsonl")
new_dataset.to_json("translated_tiny_stories.jsonl", orient="records", lines=True, force_ascii=False)

print("1000 историй переведено и сохранено.")

🔁 Переводим истории...


100%|██████████| 1000/1000 [31:59<00:00,  1.92s/it]

💾 Сохраняем в файл: translated_tiny_stories.jsonl


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

✅ Готово! 1000 историй переведено и сохранено.
